In [1]:
## Question 4.5

In [2]:
## Location 1: lambda = 3 for both returns and rentals. Poisson distribution
## Location 2: lambda = 4 for renatals, lambda = 2 for returns
## 20 cars maximum between two locations, extras are sent back to rental company
## Renting out a car gives reward of $10. Costs $2 to move a car at night
## Cars can be rented the day after they are returned
## Discount rate is .9

In [ ]:
from numpy import random
import numpy as np
from math import e, factorial

class JacksCarRental:

    def __init__(self, location_a_cars, location_b_cars, rent_reward=10, move_penalty=2, lambda_rental_a=3, 
                lambda_rental_b=4, lambda_return_a=3, lambda_return_b=2, discount=0.9):
        self.location_a_cars = location_a_cars
        self.location_b_cars = location_b_cars
        self.rent_reward = rent_reward
        self.move_penalty = move_penalty
        self.lambda_rental_a = lambda_rental_a
        self.lambda_rental_b = lambda_rental_b
        self.lambda_return_a = lambda_return_a
        self.lambda_return_b = lambda_return_b
        self.discount_rate = discount


    def get_daily_demand(self):
        a_rentals = random.poisson(lam=self.lambda_rental_a, size=1)
        b_rentals = random.poisson(lam=self.lambda_rental_b, size=1)
        a_returns = random.poisson(lam=self.lambda_return_a, size=1)
        b_returns = random.poisson(lam=self.lambda_return_b, size=1)
        return a_rentals, b_rentals, a_returns, b_returns

    
    def run_day(self):
        a_rentals, b_rentals, a_returns, b_returns = self.get_daily_demand()
        if self.location_a_cars - a_rentals < 0:
            a_rentals = self.location_a_cars
        if self.location_b_cars - b_rentals < 0:
            b_rentals = self.location_b_cars

        reward = 10 * (a_rentals + b_rentals)

        if self.location_a_cars + a_returns > 20:
            self.location_a_cars = 20
        else:
            self.location_a_cars += a_returns
        if self.location_b_cars + b_returns > 20:
            self.location_b_cars = 20
        else:
            self.location_b_cars += b_returns

        return reward


    
    def get_poission_estimator(self, lam, value):
        return e**(-lam) * lam**value / factorial(value)


    def policy_iteration(self):
        ## Initialization
        V = np.zeros((20, 20))
        pi = np.ones((20, 20))  # will define as how many cars to move from A to B

        ## Policy Evaluation
        delta = 15
        while delta < .5:
            for a in V:
                for b in a:
                    if a + b > 20:
                        continue
                    v = V[a, b]
                    V[a, b] = [self.get_poisson_estimator(self.lambda_rental_a, x) * (x * self.rent_reward + self.discount_rate * V[a - x, b - y]) for x in range(a) for y in range(b)]
